In [2]:
pip install ../QueryGPT/

Defaulting to user installation because normal site-packages is not writeable
Processing /home/laura/Dokumente/dis22-2023/QueryGPT
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for querygpt: filename=querygpt-0.0.1-py3-none-any.whl size=7807 sha256=fa27eae4673ce0106f46d20573df27d1094ec829a9bed2d53f1a10b7052712f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-ebpokk66/wheels/5e/c1/40/23fba64f43d0ab4b66ff9795059371155654e894979a7573c0
Successfully built querygpt
  Attempting uninstall: querygpt
    Found existing installation: querygpt 0.0.1
    Uninstalling querygpt-0.0.1:
      Successfully uninstalled querygpt-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
from querygpt.model import Query, Persona

## Persona selection

to use generate queries with this package we first need to select a persona to use. We can choose to add our own Persona or just load a persona from the register.  
To add a persona we can just plug in the values to the `Persona class`

In [4]:
jason = Persona(
    name="Jason",
    age=4,
    gender="male",
    job="child",
    goal="find something funny online",
    residence="paris",
    query_length=4,
    complexity=None
)

For now we are only going to load a persona from the register. This will give us all possible personas from the register.

In [5]:
Persona.list_register()

['john', 'karen', 'markus', 'edith']

To select a Persona simply use the following function.

In [6]:
person = Persona.from_register("john")

alternitavely a persona can also be loaded from a `.json` file with the `Persona.from_json()` method. The json file should look something like this:
```
{
    "name": "Edith",
    "age": 75,
    "gender": "female",
    "job": "retired",
    "goal": "researching background information",
    "residence": "suburbs of san diego",
    "query_length": 4,
    "complexity": null
}

```

## Query selection

After selecting a Persona we will need to build our query. As for the Personas we can also from a handfull of variants to load in the QueryTypes.  
- Query.select_query_type  --> loads from the register
- Query.add_custom_query_from_file --> loads from a text file
- Query.add_custom_query_from_folder --> loads from a folder containing text files
- Query.add_queries --> adds a query from a string and a QueryType name

In [7]:
q = Query()

We will first look at how to add your own query and then load one from the register.

In [8]:
query = """

Example query for $PERSONA_NAME.

"""
q.add_queries("EXAMPLE", query)

Now when we load all possible QueryTypes notice how there is one called `EXAMPLE`.

In [9]:
q.list_query_types()

dict_keys(['QUERY', 'QUERY_EXAMPLE', 'QUERY_NO_EXAMPLE', 'EXAMPLE'])

In [10]:
q.select_querie_type("QUERY_NO_EXAMPLE")

Selected query type: QUERY_NO_EXAMPLE


In [11]:
q.select_persona(person)

Selected persona: Mr. John Doe


In [12]:
q.try_query_mappings()

$PERSONA_JOB not found in query. Skipping attribute.
Attribute complexity not replaced because its value is None. Please use .manual_replace(parameter, attribute) to supply it.
Unused parameters in query: ['$PERSONA_PROFESSION', '$PERSONA_COMPLEXITY'] Please use .manual_replace(parameter, attribute) to supply them.


In [13]:
q.manual_replace("$PERSONA_PROFESSION", "lawyer")
q.manual_replace("$PERSONA_COMPLEXITY", "professional english")

In [14]:
q.query

'I want you to formulate queries about a specific topic. When writing the queries write them in the style of Mr. John Doe who wants to identify relevant document for the specific topic, to fulfill their goal of inform about legal inquiries. \nWhen creating the queries think of what Mr. John Doe is interested in given their profession lawyer. What information does Mr. John Doe need to fulfill their goal of inform about legal inquiries?. \nMake sure the queries are as realistic as possible. That means that when writing the queries use words that are realistic given that Mr. John Doe is 55 years old. \nAlso add here roughly to the length of 4 words, as it would be unrealistic for Mr. John Doe to write different length queries. \nConsider that Mr. John Doe is from new york if the topic can have a relation to new york. Formulate your words professional english. When writing the Queries also remember that Mr. John Doe is male. \nFormat the queries as a python dictionary. Write them with the 

Now we load in the core nist topics

In [15]:
import json
core_nist = json.load(open("../QueryGPT/core_nist.json"))

In [16]:
nist = core_nist[0]
nist

{'title': 'New Hydroelectric Project',
 'desc': 'Identify hydroelectric projects proposed or under construction by country and location. Detailed description of nature, extent, purpose, problems, and consequences is desirable.',
 'narr': 'Relevant documents would contain as a minimum a clear statement that a hydroelectric project is planned or construction is under way and the location of the project. Renovation of existing facilities would be judged not relevant unless plans call for a significant increase in acre-feet or reservoir or a marked change in the environmental impact of the project. Arguments for and against proposed projects are relevant as long as they are supported by specifics, including as a minimum the name or location of the project. A statement that an individual or organization is for or against such projects in general would not be relevant. Proposals or projects underway to dismantle existing facilities or drain existing reservoirs are not relevant, nor are artic

We now use the core nist topic to generate a query for the specific persona + query. `narrative` and `description` are both optional. `n_queries` defaults to 5.

In [17]:
query = q.generate_query(topic=nist["title"], description=nist["desc"], n_queries=5)
print(query)

I want you to formulate queries about a specific topic. When writing the queries write them in the style of Mr. John Doe who wants to identify relevant document for the specific topic, to fulfill their goal of inform about legal inquiries. 
When creating the queries think of what Mr. John Doe is interested in given their profession lawyer. What information does Mr. John Doe need to fulfill their goal of inform about legal inquiries?. 
Make sure the queries are as realistic as possible. That means that when writing the queries use words that are realistic given that Mr. John Doe is 55 years old. 
Also add here roughly to the length of 4 words, as it would be unrealistic for Mr. John Doe to write different length queries. 
Consider that Mr. John Doe is from new york if the topic can have a relation to new york. Formulate your words professional english. When writing the Queries also remember that Mr. John Doe is male. 
Format the queries as a python dictionary. Write them with the follow

In [20]:
import openai
openai.api_key = ""

In [21]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": query}
    ],
    temperature=0.2
    )

completion.choices[0].message

<OpenAIObject at 0x7f6d0219bab0> JSON: {
  "content": "{\n\"keywords\": [\"Hydroelectric\", \"Project\", \"Construction\"],\n\"questions\" : {\n1 : \"What are current hydroelectric projects?\",\n2 : \"Where are hydroelectric projects located?\",\n3 : \"What are the consequences of hydroelectric projects?\"\n},\n\"queries\" : {\n1 : \"Current hydroelectric projects\",\n2 : \"Location of hydroelectric projects\",\n3 : \"Consequences of hydroelectric projects\"\n}\n}\n\n{\n\"keywords\": [\"Hydroelectric\", \"Proposed\", \"Country\"],\n\"questions\" : {\n1 : \"What countries have proposed hydroelectric projects?\",\n2 : \"What are the proposed hydroelectric projects?\",\n3 : \"What is the status of proposed hydroelectric projects?\"\n},\n\"queries\" : {\n1 : \"Countries with proposed hydroelectric projects\",\n2 : \"Proposed hydroelectric projects\",\n3 : \"Status of proposed projects\"\n}\n}\n\n{\n\"keywords\": [\"Hydroelectric\", \"Extent\", \"Nature\"],\n\"questions\" : {\n1 : \"What is

In [24]:
print(completion.choices[0].message.content)

{
"keywords": ["Hydroelectric", "Project", "Construction"],
"questions" : {
1 : "What are current hydroelectric projects?",
2 : "Where are hydroelectric projects located?",
3 : "What are the consequences of hydroelectric projects?"
},
"queries" : {
1 : "Current hydroelectric projects",
2 : "Location of hydroelectric projects",
3 : "Consequences of hydroelectric projects"
}
}

{
"keywords": ["Hydroelectric", "Proposed", "Country"],
"questions" : {
1 : "What countries have proposed hydroelectric projects?",
2 : "What are the proposed hydroelectric projects?",
3 : "What is the status of proposed hydroelectric projects?"
},
"queries" : {
1 : "Countries with proposed hydroelectric projects",
2 : "Proposed hydroelectric projects",
3 : "Status of proposed projects"
}
}

{
"keywords": ["Hydroelectric", "Extent", "Nature"],
"questions" : {
1 : "What is the extent of hydroelectric projects?",
2 : "What is the nature of hydroelectric projects?",
3 : "What are the benefits of hydroelectric project

In [23]:
answer = eval(completion.choices[0].message.content)

SyntaxError: invalid syntax (<string>, line 15)

In [ ]:
answer

NameError: name 'answer' is not defined